In [ ]:
import json
from pathlib import Path
from openai import OpenAI
folder_path = Path("./json/3.개방데이터/1.데이터/Validation/02.라벨링데이터/VL_out")

client = OpenAI(api_key=openai_api_key)

In [8]:
def extract_audio_content(data:json):
    content = []
    for item in data["list"]:
        for sub_item in item["list"]:
            if "audio" in sub_item:
                content += sub_item["audio"]
    projected_content = []
    keys_to_keep = ["type", "text"]
    for item in content:
        projected_item = {k: item[k] for k in keys_to_keep if k in item}
        projected_content.append(projected_item)
    return projected_content

In [9]:
with open("data.json", "r", encoding="utf-8") as f:
    data = json.load(f)

In [10]:
len(data[0])

2

In [11]:
prompt = """
당신은 아동 발달 전문가이자 교육 상담가입니다.  
아동 상담 결과 기록이 주어지면, 해당 아동의 성향을 다음 기준으로 분석하세요:

1. 가드너의 다중지능 이론 (8가지 지능 영역)
   - 언어지능 (linguistic)
   - 논리·수학지능 (logical-mathematical)
   - 음악지능 (musical)
   - 공간지능 (spatial)
   - 신체·운동지능 (bodily-kinesthetic)
   - 대인관계지능 (interpersonal)
   - 자기성찰지능 (intrapersonal)
   - 자연탐구지능 (naturalistic)

2. 성격적 성향
   - 내향형 / 외향형 중 어느 쪽이 더 두드러지는지  

출력 형식 :
(가장 발달한 지능 유형, 내/외향형)
예시) (대인관계지능, 외향형)

상담 기록: 
{text}
"""

In [12]:
def classify(data):
    input = prompt.format(text=data["content"])
    response = client.responses.create(
        model="gpt-4o-mini", input=input, temperature=0
    )
    return response.output[0].content[0].text

In [14]:
data_concat_char = []
for item in data[:100]:
    classification = classify(item)
    data_concat_char.append({
        "id": item["id"],
        "content": item["content"],
        "classification": classification
    })

In [16]:
processed_data = []
for item in data_concat_char:
    ordered_item = {
        "id": item["id"],
        "character": item["classification"],
        "content": item["content"],
    }
    processed_data.append(ordered_item)

In [17]:
with open("data_classified.json", "w", encoding="utf-8") as f:
    json.dump(processed_data, f, ensure_ascii=False, indent=4)

In [24]:
book_prompt = """
당신은 아동 발달 전문가이자 교육 상담가입니다.  
상담 기록과 상담 기록을 통해 분석한 아동의 성향을 보고 알맞은 교재를 선택해 주세요.

1. 선택 가능한 교재의 목록
{books}

2. 상담 기록을 통해 분석된 성향
{character}

출력 형식 (교재 이름만 출력하세요) :
교재명

상담 기록: 
{text}
"""

In [19]:
with open("books.json", "r", encoding="utf-8") as f:
    books = json.load(f)

In [22]:
def classify_book(data):
    input = book_prompt.format(books=books, character=data["character"], text=data["content"])
    response = client.responses.create(model="gpt-4o-mini", input=input, temperature=0)
    return response.output[0].content[0].text

In [31]:
data_concat_book = []
for item in processed_data:
    book = classify_book(item)
    item["book"] = book
    data_concat_book.append(item)

In [32]:
with open("data_classified.json", "w", encoding="utf-8") as f:
    json.dump(data_concat_book, f, ensure_ascii=False, indent=4)

100